In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.2f}'.format
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
%matplotlib inline
from joblib import load

In [3]:
df_1 = pd.read_csv("../data/processed/df_train_1.csv", index_col=0)
df_2 = pd.read_csv("../data/processed/df_train_2.csv", index_col=0)
df_3 = pd.read_csv("../data/processed/df_train_3.csv", index_col=0)
df_4 = pd.read_csv("../data/processed/df_train_4.csv", index_col=0)
df_valid = pd.read_csv("../data/processed/df_valid.csv", index_col=0)

## log transformation on some (continuous) variables

In [4]:
# define function to preview the distribution of the (cont.) variables
def plot_cont_var_dist(df):
    cols = list(filter(lambda x: x.startswith("Variable"), df.columns))
    fig, axs = plt.subplots(len(cols)//4 + 1, 4, figsize=(20,20))
    for i, col in enumerate(cols):
        df[col].hist(ax=axs[i//4][i%4])
        axs[i//4][i%4].set_title(f"Histogram for {col}")
    plt.show()

# define function to preview the distribution of the log (cont.) variables
def plot_cont_log_var_dist(df):
    cols = list(filter(lambda x: x.startswith("Variable"), df.columns))
    fig, axs = plt.subplots(len(cols)//4 + 1, 4, figsize=(20,20))
    for i, col in enumerate(cols):
        np.log(df[col]).hist(ax=axs[i//4][i%4])
        axs[i//4][i%4].set_title(f"Histogram for log({col})")
    plt.show()

In [5]:
def log_transform(df, cols):
    df.loc[:,cols] = df.loc[:,cols].apply(lambda x: np.log(x))

## Standardization

In [6]:
# standardize the variables
sc_1 = StandardScaler()
df_1.iloc[:,6:] = sc_1.fit_transform(df_1.iloc[:,6:])
sc_2 = StandardScaler()
df_2.iloc[:,6:] = sc_2.fit_transform(df_2.iloc[:,6:])
sc_3 = StandardScaler()
df_3.iloc[:,6:] = sc_3.fit_transform(df_3.iloc[:,6:])
sc_4 = StandardScaler()
df_4.iloc[:,6:] = sc_4.fit_transform(df_4.iloc[:,6:])
sc = [sc_1, sc_2, sc_3, sc_4]

## Encoding the categorical variables

In [7]:
# drop the years in the period, only leaving Quarter, and one-hot encode
def encode_quarter(df):
    if "Period" in df.columns:
        onehotenc = OneHotEncoder(drop='first').fit(np.array(['1', '2', '3', '4']).reshape(-1, 1))
        df.loc[:,["Quarter2", "Quarter3", "Quarter4"]] = onehotenc.transform(
            df["Period"].apply(lambda x: x[-1]).to_numpy().reshape(-1, 1)
        ).toarray()
        return onehotenc
    
quarter_enc_1 = encode_quarter(df_1)
quarter_enc_2 = encode_quarter(df_2)
quarter_enc_3 = encode_quarter(df_3)
quarter_enc_4 = encode_quarter(df_4)
quarter_enc = [quarter_enc_1, quarter_enc_2, quarter_enc_3, quarter_enc_4]

In [8]:
# one-hot encode BR Code
def encode_br_code(df):
    if "BR Code" in df.columns:
        onehotenc = OneHotEncoder(drop='first').fit(np.array(range(108)).reshape(-1, 1))
        df.loc[:,[f"BR Code {c}" for c in range(1, 108)]] = onehotenc.transform(
            df["BR Code"].to_numpy().reshape(-1, 1)
        ).toarray()
        return onehotenc
    
br_enc_1 = encode_br_code(df_1)
br_enc_2 = encode_br_code(df_2)
br_enc_3 = encode_br_code(df_3)
br_enc_4 = encode_br_code(df_4)
br_enc = [br_enc_1, br_enc_2, br_enc_3, br_enc_4]

In [9]:
# one-hot encode Country Code
def encode_country_code(df):
    if "Country_Code" in df.columns:
        onehotenc = OneHotEncoder(drop='first').fit(np.array([0, 1, 2]).reshape(-1, 1))
        df.loc[:,["Country_Code_1", "Country_Code_2"]] = onehotenc.transform(
            df["Country_Code"].to_numpy().reshape(-1, 1)
        ).toarray()
        return onehotenc
    
country_enc_1 = encode_country_code(df_1)
country_enc_2 = encode_country_code(df_2)
country_enc_3 = encode_country_code(df_3)
country_enc_4 = encode_country_code(df_4)
country_enc = [country_enc_1, country_enc_2, country_enc_3, country_enc_4]

In [10]:
# encode the outcome, risk rating
risk_rating_enc = LabelEncoder()
risk_rating_enc.fit(list(range(1, 16)) + [17])
df_1["risk_rating"] = risk_rating_enc.transform(df_1["risk_rating"])
df_2["risk_rating"] = risk_rating_enc.transform(df_2["risk_rating"])
df_3["risk_rating"] = risk_rating_enc.transform(df_3["risk_rating"])
df_4["risk_rating"] = risk_rating_enc.transform(df_4["risk_rating"])

In [11]:
# remove encoded and useless columns
df_1 = df_1.drop(columns=["Country_Code", "BR Code", "Period", "Client"])
df_2 = df_2.drop(columns=["Country_Code", "BR Code", "Period", "Client"])
df_3 = df_3.drop(columns=["Country_Code", "BR Code", "Period", "Client"])
df_4 = df_4.drop(columns=["Country_Code", "BR Code", "Period", "Client"])

## Comparing the 4 datasets

#### Prepare the validation datasets

In [12]:
# examine the four different training datasets (with different missing data handling)
# by fitting a simple decision tree model and test.

# load the files for feature selection and featuer engineering on the validation dataset
with open("../references/mean_sd.npy", "rb") as f:
    mean, sd = np.load(f)
simple_imputer = load("../references/simple_imputer.joblib")
reg_imputer = load("../references/regression_imputer.joblib")
selected_cols = []
with open("../references/selected_features.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        selected_cols.append(df_valid.columns[:6].to_list() + line.strip().split(","))

# apply the same manipulations on the validation dataset to obtain the four sets of validation 
# data for the four training datasets
def transform_and_encode(df, df_ind):
    df_ind -= 1
    df.iloc[:,6:] = sc[df_ind].transform(df.iloc[:,6:])
    df.loc[:,["Quarter2", "Quarter3", "Quarter4"]] = quarter_enc[df_ind].transform(
            df["Period"].apply(lambda x: x[-1]).to_numpy().reshape(-1, 1)).toarray()
    df.loc[:,[f"BR Code {c}" for c in range(1, 108)]] = br_enc[df_ind].transform(
            df["BR Code"].to_numpy().reshape(-1, 1)).toarray()
    df.loc[:,["Country_Code_1", "Country_Code_2"]] = country_enc[df_ind].transform(
            df["Country_Code"].to_numpy().reshape(-1, 1)).toarray()
    df.loc[:,"risk_rating"] = risk_rating_enc.transform(df["risk_rating"])
    return df.drop(columns=["Country_Code", "BR Code", "Period", "Client"])

df_valid = df_valid.iloc[(~(np.abs(df_valid.iloc[:,6:] - mean) > sd * 2).any(axis=1)).to_list(),:]

df_valid_1 = df_valid.loc[:,selected_cols[0]].dropna().copy()
df_valid_1 = transform_and_encode(df_valid_1, 1)

df_valid_2 = df_valid.loc[:,selected_cols[1]].dropna().copy()
df_valid_2 = transform_and_encode(df_valid_2, 2)

df_valid_3 = df_valid.copy()
df_valid_3.iloc[:,6:] = simple_imputer.transform(df_valid_3.iloc[:,6:])
df_valid_3 = df_valid_3.loc[:,selected_cols[2]]
df_valid_3 = transform_and_encode(df_valid_3, 3)

df_valid_4 = df_valid.copy()
df_valid_4.iloc[:,6:] = reg_imputer.transform(df_valid_4.iloc[:,6:])
df_valid_4 = df_valid_4.loc[:,selected_cols[3]]
df_valid_4 = transform_and_encode(df_valid_4, 4)


[IterativeImputer] Completing matrix with shape (4444, 118)
[IterativeImputer] Ending imputation round 1/18, elapsed time 0.26
[IterativeImputer] Ending imputation round 2/18, elapsed time 0.53
[IterativeImputer] Ending imputation round 3/18, elapsed time 0.80
[IterativeImputer] Ending imputation round 4/18, elapsed time 1.03
[IterativeImputer] Ending imputation round 5/18, elapsed time 1.26
[IterativeImputer] Ending imputation round 6/18, elapsed time 1.50
[IterativeImputer] Ending imputation round 7/18, elapsed time 1.74
[IterativeImputer] Ending imputation round 8/18, elapsed time 1.98
[IterativeImputer] Ending imputation round 9/18, elapsed time 2.20
[IterativeImputer] Ending imputation round 10/18, elapsed time 2.46
[IterativeImputer] Ending imputation round 11/18, elapsed time 2.68
[IterativeImputer] Ending imputation round 12/18, elapsed time 2.93
[IterativeImputer] Ending imputation round 13/18, elapsed time 3.18
[IterativeImputer] Ending imputation round 14/18, elapsed time 3.

#### Use a simple decision tree model to assess the datasets

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
clf_1 = DecisionTreeClassifier(random_state=821)
clf_1.fit(df_1.iloc[:,2:], df_1["risk_rating"])
y_pred_1 = clf_1.predict(df_valid_1.iloc[:,2:])
print(f"model with 1st dataset: {f1_score(df_valid_1['risk_rating'], y_pred_1, average='micro')}")

clf_2 = DecisionTreeClassifier(random_state=821)
clf_2.fit(df_2.iloc[:,2:], df_2["risk_rating"])
y_pred_2 = clf_2.predict(df_valid_2.iloc[:,2:])
print(f"model with 2nd dataset: {f1_score(df_valid_2['risk_rating'], y_pred_2, average='micro')}")

clf_3 = DecisionTreeClassifier(random_state=821)
clf_3.fit(df_3.iloc[:,2:], df_3["risk_rating"])
y_pred_3 = clf_3.predict(df_valid_3.iloc[:,2:])
print(f"model with 3rd dataset: {f1_score(df_valid_3['risk_rating'], y_pred_3, average='micro')}")

clf_4 = DecisionTreeClassifier(random_state=821)
clf_4.fit(df_4.iloc[:,2:], df_4["risk_rating"])
y_pred_4 = clf_4.predict(df_valid_4.iloc[:,2:])
print(f"model with 4th dataset: {f1_score(df_valid_4['risk_rating'], y_pred_4, average='micro')}")

model with 1st dataset: 0.17087967644084934
model with 2nd dataset: 0.16594110115236876
model with 3rd dataset: 0.1707920792079208
model with 4th dataset: 0.16741674167416742


The last dataset (4th) gives the best f1 score, which measures the overall performance (precision and recall) of the model fitted. Here, `average='micro'` is used for evaluating the overall f1 score globally (instead of averaging the individual f1 score for each label), so that the weight of each label is considered.

The 4th dataset (together with the corresponding method of handling missing data and selected features) is selected to perform the next step - model fitting and tuning.

In [15]:
# manipulate the test dataset in the same way as the 4th train/validation dataset, 
# and save all of them as the finalized train/validation/test datasets.
df_test = pd.read_csv("../data/processed/df_test.csv", index_col=0)
df_test = df_test.iloc[(~(np.abs(df_test.iloc[:,6:] - mean) > sd * 2).any(axis=1)).to_list(),:]
df_test.iloc[:,6:] = reg_imputer.transform(df_test.iloc[:,6:])
df_test = df_test.loc[:,selected_cols[3]]
df_test = transform_and_encode(df_test, 4)

df_4.to_csv("../data/processed/df_train_final.csv")
df_valid_4.to_csv("../data/processed/df_valid_final.csv")
df_test.to_csv("../data/processed/df_test_final.csv")

[IterativeImputer] Completing matrix with shape (4451, 118)
[IterativeImputer] Ending imputation round 1/18, elapsed time 0.25
[IterativeImputer] Ending imputation round 2/18, elapsed time 0.47
[IterativeImputer] Ending imputation round 3/18, elapsed time 0.69
[IterativeImputer] Ending imputation round 4/18, elapsed time 0.95
[IterativeImputer] Ending imputation round 5/18, elapsed time 1.20
[IterativeImputer] Ending imputation round 6/18, elapsed time 1.44
[IterativeImputer] Ending imputation round 7/18, elapsed time 1.67
[IterativeImputer] Ending imputation round 8/18, elapsed time 1.91
[IterativeImputer] Ending imputation round 9/18, elapsed time 2.14
[IterativeImputer] Ending imputation round 10/18, elapsed time 2.37
[IterativeImputer] Ending imputation round 11/18, elapsed time 2.61
[IterativeImputer] Ending imputation round 12/18, elapsed time 2.85
[IterativeImputer] Ending imputation round 13/18, elapsed time 3.09
[IterativeImputer] Ending imputation round 14/18, elapsed time 3.